In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/df_wine_clean_no.csv')

In [3]:
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,title,...,130,131,132,133,134,135,136,137,138,139
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,39.5,Sicily & Sardinia,Etna,None,Nicosia 2013 Vulkà Bianco (Etna),...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,None,None,Quinta dos Avidagos 2011 Avidagos Red (Douro),...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2,2,US,"Tart and snappy, the flavors of lime flesh and...",None,87,14.0,Oregon,Willamette Valley,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),...,0.0,0.0,0.0,0.081144,0.000000,0.000000,0.0,0.0,0.0,0.0
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,None,St. Julian 2013 Reserve Late Harvest Riesling ...,...,0.0,0.0,0.0,0.349908,0.092807,0.000000,0.0,0.0,0.0,0.0
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,...,0.0,0.0,0.0,0.000000,0.000000,0.048212,0.0,0.0,0.0,0.0


In [7]:
from nltk.corpus import stopwords
from nltk import download
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
import gensim.downloader as api
from gensim.similarities import SparseTermSimilarityMatrix, WordEmbeddingSimilarityIndex
import re

In [23]:
def clean(text):
    # replace any non-letters with a space
    text = re.sub('[^A-Za-z]+', ' ', text)
    # lower case the text
    text = text.lower()
    return text

def token_stop(text):
    temp = clean(text)
    tokens = word_tokenize(temp) # tokenize the text
    newlist = []
    for word in tokens:
        if word not in set(stopwords.words('english')): # remove stop words
            newlist.append(word)
    return newlist

In [9]:
import ast
df['LDA description'] = [ast.literal_eval(i) for i in df['LDA description']]

In [10]:
dict_ = Dictionary(df['LDA description'])

In [16]:
doc_term_matrix = [dict_.doc2bow(i) for i in df['LDA description']]

In [26]:
tfidf = TfidfModel(doc_term_matrix)

In [60]:
fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')

[=================================================-] 99.9% 957.9/958.4MB downloaded


In [61]:
termsim_index = WordEmbeddingSimilarityIndex(fasttext_model300)
termsim_matrix = SparseTermSimilarityMatrix(termsim_index, dict_, tfidf)


100%|███████████████████████████████████| 30255/30255 [29:32<00:00, 17.06it/s]


In [117]:
termsim_matrix.save('termsim_matrix')

In [119]:
termsim_matrix = SparseTermSimilarityMatrix.load('termsim_matrix')

# Soft Cosine Measure

In [11]:
dict_ = Dictionary(df['LDA description'])
termsim_matrix = SparseTermSimilarityMatrix.load('data/termsim_matrix')

In [12]:
new_input = "Grown on clay-loam soils, with tough ironstone rocks running through, this wine is the most savory and brambly of Giant Steps's single-vineyard Pinots.A thicket of red and blueberry fruit entwines with white pepper and other savory spices, dried rosesand meaty bass notes. In the mouth, fleshy fruit is again met with lovely spice, cinched by fine tannins.There's freshness, structure and elegance here that makes this a complete wine now, but one that could cellar until around 2032."


In [24]:
def soft_cosine_measure_similarity(new_input:str, termsim_matrix, dictionary, k=10, sort_points=False):
    new_vector = tfidf[dictionary.doc2bow(token_stop(new_input))]
    similarity_scores = []
    for i in range(len(vectors)):
        similarity = termsim_matrix.inner_product(new_vector, vectors[i], normalized=(True, True))
        similarity_scores.append((i,similarity))
    top_k = sorted(similarity_scores, key=lambda x: x[1], reverse = True)[:k]
    top_k_index = [tuple_[0] for tuple_ in top_k]
    title_points = []
    for index in top_k_index:
        title_points.append((df.iloc[index]['title'], round(df.iloc[index]['normalized rating'],2)))
    if sort_points:
        print(f'{k} Most similar wines (descending order by similarity and points):')
        sorted_title_points = sorted(title_points, key=lambda x: x[1], reverse = True)
        for i, tuple_ in enumerate(sorted_title_points):
            print(f'{i+1}. {tuple_[0]} ---- {tuple_[1]}')
    else:   
        print(f'{k} Most similar wines (descending order by similarity):')
        for i, tuple_ in enumerate(title_points):
            print(f'{i+1}. {tuple_[0]} ---- {tuple_[1]}')

In [25]:
soft_cosine_measure_similarity(new_input, termsim_matrix, dict_, sort_points = True)

NameError: name 'vectors' is not defined